In [12]:
import numpy as np
import torch
from torchvision.datasets import mnist # 导入 pytorch 内置的 mnist 数据
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torch.nn.functional as F     # 激励函数
from torch import nn
import torch.utils.data as Data
import time
from torchvision import datasets, transforms

In [35]:
def data_tf(x):
#     print(np.array(x, dtype='float32'))
    x = np.array(x, dtype='float32') /255
    x = (x - 0.5) / 0.5 # 标准化
    x = x.reshape((-1,)) # 拉平
    x = torch.Tensor(x)
    return x

train_dataset = mnist.MNIST('./data', train=True, transform=data_tf, download=True) # 重新载入数据集，申明定义的数据变换
test_dataset = mnist.MNIST('./data', train=False, transform=data_tf, download=True)
print(len(train_dataset), len(test_dataset))
a_img, a_label = train_dataset[0]
print(a_img.shape)
print(a_label)
train_data = Data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_data = Data.DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

a, a_label = next(iter(train_data))
# 打印出一个批次的数据大小
print(a.shape)
print(a_label.shape)

60000 10000
torch.Size([784])
5
torch.Size([64, 784])
torch.Size([64])


In [33]:
# 使用 Sequential 定义神经网络
net = nn.Sequential(
    nn.Linear(784, 400),
    nn.ReLU(),
    nn.Linear(400, 100),
    nn.ReLU(),
    nn.Linear(100, 10)
)

print(net)

# 定义 loss 函数
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), 1e-1) # 使用随机梯度下降，学习率 0.1

acc_train_list = []
loss_train_list = []
acc_test_list = []
loss_test_list = []

for epoch in range(20):
    start = time.time()
    # 开始训练
    acc_train = 0.0
    loss_train = 0.0
    acc_test = 0.0
    loss_test = 0.0
    net.train()
    for x_train, y_train in train_data:
        y_out = net(x_train)
        loss = criterion(y_out, y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, y_pred_train = y_out.max(1)
        # print(y_out[0])
        # print(y_pred[0])
        acc_train += np.sum(y_pred_train.numpy() == y_train.numpy())
        loss_train += loss.data.numpy()*x_train.shape[0]
    acc_train_list.append(acc_train/len(train_dataset))
    loss_train_list.append(loss_train/len(train_dataset))

    net.eval()
    for x_test, y_test in test_data:
        y_test_out = net(x_test)
        loss2 = criterion(y_test_out, y_test)
        _, y_pred_test = y_test_out.max(1)
        acc_test += np.sum(y_pred_test.numpy() == y_test.numpy())
        loss_test += loss2.data.numpy()*x_test.shape[0]
    acc_test_list.append(acc_test / len(test_dataset))
    loss_test_list.append(loss_test / len(test_dataset))

    time_elapsed = time.time() - start
    print('epoch[%d/20]:训练集loss[%.4f], acc[%.4f].测试集loss[%.4f], acc[%.4f]' % (epoch, loss_train/len(train_dataset), acc_train/len(train_dataset),
                                                                 loss_test / len(test_dataset), acc_test / len(test_dataset)))
    print('Epoch training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))



Sequential(
  (0): Linear(in_features=784, out_features=400, bias=True)
  (1): ReLU()
  (2): Linear(in_features=400, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=10, bias=True)
)
epoch[0/20]:训练集loss[0.4259], acc[0.8659].测试集loss[0.2152], acc[0.9321]
Epoch training complete in 0m 7s
epoch[1/20]:训练集loss[0.1574], acc[0.9515].测试集loss[0.1385], acc[0.9565]
Epoch training complete in 0m 7s
epoch[2/20]:训练集loss[0.1137], acc[0.9650].测试集loss[0.1033], acc[0.9684]
Epoch training complete in 0m 7s
epoch[3/20]:训练集loss[0.0883], acc[0.9718].测试集loss[0.1032], acc[0.9688]
Epoch training complete in 0m 7s
epoch[4/20]:训练集loss[0.0717], acc[0.9776].测试集loss[0.0795], acc[0.9762]
Epoch training complete in 0m 7s
epoch[5/20]:训练集loss[0.0597], acc[0.9805].测试集loss[0.0740], acc[0.9768]
Epoch training complete in 0m 7s


KeyboardInterrupt: 

In [ ]:

t = np.arange(1,21)
plt.figure(1)
plt.subplot(1,2,1)
plt.plot(t, np.array(acc_train_list), 'b-', label='train')
plt.plot(t, np.array(acc_test_list), 'r-', label='test')
plt.title("Accuracy")
plt.legend()
plt.subplot(1,2,2)
plt.plot(t, np.array(loss_train_list), 'b-', label='train')
plt.plot(t, np.array(loss_test_list), 'r-', label='test')
plt.title("Loss")
plt.legend()
plt.show()